In [3]:
import pandas as pd
import random
import csv
import os
import math

traindf = pd.read_pickle("./pickles/df_small_train.pkl")
testdf = pd.read_pickle("./pickles/df_small_test.pkl")

In [4]:
def random_solution(df, csv_name):
    
    csv_name = "./results/" + csv_name
    
    # get list of hotels per search id
    results_per_search = df.groupby('srch_id').size()
    
    column_values = df["srch_id"].values
    ids =  pd.unique(column_values) 
    
    if not os.path.exists("./results"):
        os.mkdir("./results")
        
    with open(csv_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["srch_id", "prop_id"])

    for i, ideetje in enumerate(ids):
        df_temp = df[df["srch_id"] == ideetje]
        prop_ids = df_temp["prop_id"].values
        
        # shuffle hotels
        random.shuffle(prop_ids)
    
        # write csv
        with open(csv_name, 'a', newline='') as file:
            writer = csv.writer(file)
            
            for prop_id in prop_ids:
                writer.writerow([ideetje, prop_id])

In [5]:
def price_quality(df):

    df['pricequality'] = df['price_usd'] / df['prop_starrating']
    
    return df

In [12]:
from tqdm import tqdm

def price_quality_solution(df, csv_name):
    
    csv_name = "./results/" + csv_name
    
    # get list of hotels per search id
    results_per_search = df.groupby('srch_id').size()
    
    column_values = df["srch_id"].values
    ids =  pd.unique(column_values) 
    
    if not os.path.exists("./results"):
        os.mkdir("./results")
        
    with open(csv_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["srch_id", "prop_id"])

    total_ids = len(ids)    
        
    for i in tqdm(range(len(ids))):
        ideetje = ids[i]
        
        # get all rows that belong with this search
        df_temp = df[df["srch_id"] == ideetje]
        
        # get list of property ids
        prop_ids = df_temp["prop_id"].values
        
        # rank the property ids based on price/quality
        prop_ids_sorted = [x for _, x in sorted(zip(df_temp["pricequality"], prop_ids))]
                
        # write csv
        with open(csv_name, 'a', newline='') as file:
            writer = csv.writer(file)
            
            for prop_id in prop_ids_sorted:
                writer.writerow([ideetje, prop_id])

In [13]:
testdf = price_quality(testdf)
price_quality_solution(df=testdf, csv_name="price_quality_sol_small.csv")

100%|████████████████████████████████████████████████████████████████████████████| 1996/1996 [00:01<00:00, 1005.10it/s]


In [8]:
def change_time(df):
    
    if 'date_time' in df.columns:
        # Change time information to year and month columns
        df["date_time"] = pd.to_datetime(df["date_time"])
        df["year"] = df["date_time"].dt.year
        df["month"] = df["date_time"].dt.month

        #Delete column date-time
        df = df.drop('date_time', 1)
        
    return df

In [12]:
X = ["0", "1", "3", "20", "8"]
Y = [0, 1, 3, 20, 8]


In [13]:
srtd = [x for _,x in sorted(zip(Y,X))]
print(srtd)

['0', '1', '3', '8', '20']


1587981335.8008757
